In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import tensorflow as tf

In [6]:
df = pd.read_csv('/content/email_classification2.csv')

In [7]:
df.head()

,text,label
0,sentto jm jmason org groups yahoo com mon dec ...,0.0
1,ilug admin ie mon sep return path delivered zz...,0.0
2,return path gward net delivery date mon sep gw...,0.0
3,ilug admin ie mon jul return path delivered yy...,0.0
4,rpm list admin net fri aug return path deliver...,0.0


In [4]:
from bs4 import BeautifulSoup
import re
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords

tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def email_body_extractor(text):
    # Use BeautifulSoup to parse HTML content
    soup = BeautifulSoup(text, 'lxml')
    # Extract the text content of the email
    email_body = soup.get_text()
    # Remove email addresses and URLs
    stripped = re.sub(combined_pat, '', email_body)
    try:
        # Handle utf-8 encoding issues
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    # Remove non-alphabetic characters
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    # Convert to lowercase
    lower_case = letters_only.lower()
    # Tokenize and remove stopwords
    words = tok.tokenize(lower_case)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    # Join the words back into a string
    return (" ".join(words)).strip()


testing = df.text[:10]
test_result = [email_body_extractor(t) for t in testing]
print(test_result)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
nums = [0,5795]
print("Cleaning and parsing the emails...\n")
clean_tweet_texts = []
for i in range(nums[0],nums[1]):
    if( (i+1)%100 == 0 ):
        print("Emails %d of %d has been processed" % ( i+1, nums[1] ))
    clean_tweet_texts.append(email_body_extractor(df['text'][i]))

Cleaning and parsing the tweets...

Tweets 100 of 5795 has been processed
Tweets 200 of 5795 has been processed
Tweets 300 of 5795 has been processed
Tweets 400 of 5795 has been processed
Tweets 500 of 5795 has been processed
Tweets 600 of 5795 has been processed
Tweets 700 of 5795 has been processed
Tweets 800 of 5795 has been processed
Tweets 900 of 5795 has been processed
Tweets 1000 of 5795 has been processed
Tweets 1100 of 5795 has been processed
Tweets 1200 of 5795 has been processed
Tweets 1300 of 5795 has been processed
Tweets 1400 of 5795 has been processed
Tweets 1500 of 5795 has been processed
Tweets 1600 of 5795 has been processed
Tweets 1700 of 5795 has been processed
Tweets 1800 of 5795 has been processed
Tweets 1900 of 5795 has been processed
Tweets 2000 of 5795 has been processed
Tweets 2100 of 5795 has been processed
Tweets 2200 of 5795 has been processed
Tweets 2300 of 5795 has been processed
Tweets 2400 of 5795 has been processed
Tweets 2500 of 5795 has been processe

In [ ]:
clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])
clean_df['label'] = df.target
clean_df.head()

,text,label
0,ilug admin ie mon jul return path delivered yy...,0
1,gort com mon jun return path gort com delivery...,1
2,fork admin com mon jul return path delivered y...,1
3,dcm net cn mon jun return path dcm net cn deli...,1
4,ilug admin ie mon aug return path delivered yy...,0


In [ ]:
clean_df.to_csv('email_classification.csv',encoding='utf-8')
df = pd.read_csv('email_classification.csv')

In [ ]:
df.head()

,Unnamed: 0,text,label
0,0,ilug admin ie mon jul return path delivered yy...,0
1,1,gort com mon jun return path gort com delivery...,1
2,2,fork admin com mon jul return path delivered y...,1
3,3,dcm net cn mon jun return path dcm net cn deli...,1
4,4,ilug admin ie mon aug return path delivered yy...,0


In [ ]:
df = pd.read_csv('email_classification.csv')

print("Original DataFrame:")
print(df)

# Find and drop unnamed columns
unnamed_columns = [col for col in df.columns if 'Unnamed' in col]
df = df.drop(columns=unnamed_columns, errors='ignore')

# Display the DataFrame after dropping unnamed columns
print("\nDataFrame after dropping unnamed columns:")
print(df)


Original DataFrame:
      Unnamed: 0                                               text  label
0              0  ilug admin ie mon jul return path delivered yy...      0
1              1  gort com mon jun return path gort com delivery...      1
2              2  fork admin com mon jul return path delivered y...      1
3              3  dcm net cn mon jun return path dcm net cn deli...      1
4              4  ilug admin ie mon aug return path delivered yy...      0
...          ...                                                ...    ...
5790        5790  crackmice admin com thu sep return path delive...      1
5791        5791  ilug admin ie mon jul return path delivered yy...      0
5792        5792  fork admin com mon oct return path delivered y...      0
5793        5793  received hq pro ns net localhost hq pro ns net...      1
5794        5794  razor users admin sourceforge net thu sep retu...      0

[5795 rows x 3 columns]

DataFrame after dropping unnamed columns:
            

In [ ]:
input_file = 'email_classification.csv'
output_file = 'email_classification1.csv'

df = pd.read_csv(input_file)

print("Original DataFrame:")
print(df)

unnamed_columns = [col for col in df.columns if 'Unnamed' in col]
df = df.drop(columns=unnamed_columns, errors='ignore')

print("\nDataFrame after dropping unnamed columns:")
print(df)

df.to_csv(output_file, index=False)

print(f"\nModified DataFrame saved to {output_file}")


Original DataFrame:
      Unnamed: 0                                               text  label
0              0  ilug admin ie mon jul return path delivered yy...      0
1              1  gort com mon jun return path gort com delivery...      1
2              2  fork admin com mon jul return path delivered y...      1
3              3  dcm net cn mon jun return path dcm net cn deli...      1
4              4  ilug admin ie mon aug return path delivered yy...      0
...          ...                                                ...    ...
5790        5790  crackmice admin com thu sep return path delive...      1
5791        5791  ilug admin ie mon jul return path delivered yy...      0
5792        5792  fork admin com mon oct return path delivered y...      0
5793        5793  received hq pro ns net localhost hq pro ns net...      1
5794        5794  razor users admin sourceforge net thu sep retu...      0

[5795 rows x 3 columns]

DataFrame after dropping unnamed columns:
            

In [ ]:
df = pd.read_csv('email_classification1.csv')

In [ ]:
df.head()

,text,label
0,ilug admin ie mon jul return path delivered yy...,0
1,gort com mon jun return path gort com delivery...,1
2,fork admin com mon jul return path delivered y...,1
3,dcm net cn mon jun return path dcm net cn deli...,1
4,ilug admin ie mon aug return path delivered yy...,0


In [ ]:
df.label.value_counts()

label
0    3899
1    1896
Name: count, dtype: int64

In [ ]:
df = pd.read_csv('email_classification1.csv')

classes = df['label'].unique()

samples_per_class = 1896

selected_data = pd.DataFrame()

for class_label in classes:

    class_data = df[df['label'] == class_label].sample(samples_per_class, random_state=42)


    selected_data = selected_data._append(class_data)

selected_data.to_csv('email_classification2.csv', index=False)

In [ ]:
df = pd.read_csv('email_classification2.csv')
df.label.value_counts()

label
0    1896
1    1896
Name: count, dtype: int64

In [12]:
pip install tensorflow-data-validation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.5/22.5 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
import pandas as pd
import tensorflow_data_validation as tfdv
df = pd.read_csv('email_classification2.csv')

In [14]:
statistics = tfdv.generate_statistics_from_dataframe(df)

schema = tfdv.infer_schema(statistics)

In [15]:
anomalies = tfdv.validate_statistics(statistics, schema)

#anomalies
tfdv.display_anomalies(anomalies)

In [16]:
#feature statistics
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'text',BYTES,required,,-
'label',INT,required,,-


In [18]:
# Visualize data statistics
tfdv.visualize_statistics(lhs_statistics=statistics, lhs_name='Dataset')